In [ ]:
import influxdb_client
import pandas as pd
import hashlib
import numpy as np
import networkx as nx
import time
import subprocess
import warnings
from influxdb_client.client.write_api import SYNCHRONOUS
from IPython.display import clear_output
from influxdb_client.client.warnings import MissingPivotFunction
from datetime import datetime, timedelta

warnings.simplefilter("ignore", MissingPivotFunction)


# InfluxDB instance details
bucket = "schempp"
org = "UMA"
token = "TOKEN_ID"
url="http://IP_ADDRESS:PORT"
# Create influxDB client connection
client = influxdb_client.InfluxDBClient(url=url,token=token,org=org, debug=False)

# Query API
query_api = client.query_api()

# Dataframes creation
automaton_df = pd.DataFrame(columns=['configID', 'qtyID', 'transition', 'slapsed_time'])
total_configs_df = pd.DataFrame(columns=['dlMaxRetxThreshold', 'dlPollByte', 'dlPollPDU', 'dlTPollRetr', 'dlTProhib', 'dlTReassembly', 'ulMaxRetxThreshold', 'ulPollByte', 'ulPollPDU', 'ulTPollRetr', 'ulTProhib', 'ulTReassembly','id'])
reconfigurations_df = pd.DataFrame(columns=['original_configID', 'selected_configID', 'time'])
# Variables
current_config_id, current_qty_id, current_state_id = (0,)*3
old_config_id = 0
read_config_id, read_qty_id, read_state_id = (0,)*3
time_state = 0
config_time = 15
current_time = ""
revisited_state = False

# Functions definition

def addReconfiguration(original_configID, selected_configID, time):
    global reconfigurations_df
    reconfigurations_df = reconfigurations_df.append({"original_configID":original_configID, "selected_configID":selected_configID, "time":time}, ignore_index=True)

def getqtyID(delay, jitter):
    global qty_df
    for i in range(len(qty_df)):
        if (qty_df.iloc[i]['min_delay'] <= delay and
            qty_df.iloc[i]['max_delay'] >= delay and
            qty_df.iloc[i]['min_jitter'] <= jitter and
            qty_df.iloc[i]['max_jitter'] >= jitter):
            return qty_df.iloc[i]['qty']
    return 0


def addConfig(configParams, configID):
    global total_configs_df
    global current_config_id
    if (configExists(configID) == False):
        configToAppend = {"dlMaxRetxThreshold":configParams[0], "dlPollByte":configParams[1],
                            "dlPollPDU":configParams[2], "dlTPollRetr":configParams[3],
                            "dlTProhib":configParams[4], "dlTReassembly":configParams[5],
                            "ulMaxRetxThreshold":configParams[6], "ulPollByte":configParams[7],
                            "ulPollPDU":configParams[8], "ulTPollRetr":configParams[9],
                            "ulTProhib":configParams[10], "ulTReassembly":configParams[11],
                            "id":configID}

        total_configs_df = total_configs_df.append(configToAppend, ignore_index=True)


def getConfigID(configToCheck):
    global total_configs_df
    for i in range(len(total_configs_df)):
        if (total_configs_df.iloc[i]['dlMaxRetxThreshold'] == configToCheck['dlMaxRetxThreshold'] and
            total_configs_df.iloc[i]['dlPollByte'] == configToCheck['dlPollByte'] and
            total_configs_df.iloc[i]['dlPollPDU'] == configToCheck['dlPollPDU'] and
            total_configs_df.iloc[i]['dlTPollRetr'] == configToCheck['dlTPollRetr'] and
            total_configs_df.iloc[i]['dlTProhib'] == configToCheck['dlTProhib'] and
            total_configs_df.iloc[i]['dlTReassembly'] == configToCheck['dlTReassembly']):
            return i
    return -1

def configExists(configID):
    global total_configs_df
    for i in range(len(total_configs_df)):
        if (total_configs_df.iloc[i]['id'] == configID):
            return True
    return False

def getConfigParams(configID):
    global total_configs_df
    for i in range(len(total_configs_df)):
        if (total_configs_df.iloc[i]['id'] == configID):
            return total_configs_df.iloc[i]
    return -1

    
def manageAutomatonState(readState):
    #Global variables
    global automaton_df
    global current_config_id
    global current_qty_id
    global current_state_id
    global time_state
    global revisited_state
        
    #The automaton is empty
    if (automaton_df.empty):
        #Add the state to the automaton
        automaton_df = automaton_df.append(readState, ignore_index=True)
        print("First state added to the automaton at " + current_time + " (current state: " + str(current_state_id) + ")")
        #Update the global variables
        current_config_id = readState['configID']
        current_qty_id = readState['qtyID']
        current_state_id = automaton_df.tail(1).index.item()
        time_state = 0
        return current_state_id
    
    #The automaton is not empty
    else:
        #Check if the state is already in the automaton
        automaton_index = getIndexAutomatonStateByConfigIDAndQtyID(readState['configID'], readState['qtyID'])

        #The state is in the automaton and is not the current state --> add transition type
        if (automaton_index != -1 and automaton_index != current_state_id):
            if (readState['configID'] != current_config_id and readState['qtyID'] != current_qty_id):
                if ((automaton_df.at[current_state_id, 'transition'][0]) == ""):
                    automaton_df.at[current_state_id, 'transition'][0] = ("config_qty-" + str(automaton_index))
                else:
                    automaton_df.at[current_state_id, 'transition'].append("config_qty-" + str(automaton_index) )
            elif (readState['configID'] != current_config_id):
                if ((automaton_df.at[current_state_id, 'transition'][0]) == ""):
                    automaton_df.at[current_state_id, 'transition'][0] = ("config-" + str(automaton_index))
                else:
                    automaton_df.at[current_state_id, 'transition'].append("config-" + str(automaton_index))
            elif (readState['qtyID'] != current_qty_id):
                if ((automaton_df.at[current_state_id, 'transition'][0]) == ""):
                    automaton_df.at[current_state_id, 'transition'][0] = ("qty-" + str(automaton_index))
                else:
                    automaton_df.at[current_state_id, 'transition'].append("qty-" + str(automaton_index))
            if (automaton_df.at[current_state_id, 'slapsed_time'][0] == 0):
                automaton_df.at[current_state_id, 'slapsed_time'][0] = time_state
            else:    
                automaton_df.at[current_state_id, 'slapsed_time'].append(time_state)
            #Update the global variables
            current_config_id = readState['configID']
            current_qty_id = readState['qtyID']
            current_state_id = automaton_index
            time_state = 0
            revisited_state = True
            return current_state_id

        elif (automaton_index != -1 and automaton_index == current_state_id):
            #Do nothing
            return current_state_id

        #The state is NOT in the automaton
        else:
            #Check and add transition type
            if (current_config_id != readState['configID'] and current_config_id != readState['qtyID']):
                if ((automaton_df.at[current_state_id, 'transition'][0]) == ""):
                    automaton_df.at[current_state_id, 'transition'][0] = ("config_qty-" + str(automaton_df.tail(1).index.item()+1))
                else:
                    automaton_df.at[current_state_id, 'transition'].append("config_qty-" + str(automaton_df.tail(1).index.item()+1))
            elif (current_config_id != readState['configID']):
                if ((automaton_df.at[current_state_id, 'transition'][0]) == ""):
                    automaton_df.at[current_state_id, 'transition'][0] = ("config-" + str(automaton_df.tail(1).index.item()+1))
                else:
                    automaton_df.at[current_state_id, 'transition'].append("config-" + str(automaton_df.tail(1).index.item()+1))
            elif (current_config_id != readState['qtyID']):
                if ((automaton_df.at[current_state_id, 'transition'][0]) == ""):
                    automaton_df.at[current_state_id, 'transition'][0] = ("qty-" + str(automaton_df.tail(1).index.item()+1))
                else:
                    automaton_df.at[current_state_id, 'transition'].append("qty-" + str(automaton_df.tail(1).index.item()+1))
            if (automaton_df.at[current_state_id, 'slapsed_time'][0] == 0):
                automaton_df.at[current_state_id, 'slapsed_time'][0] = time_state
            else:    
                automaton_df.at[current_state_id, 'slapsed_time'].append(time_state)
            #Add the state to the automaton
            automaton_df = automaton_df.append(readState, ignore_index=True)
            #Update the global variables
            current_config_id = readState['configID']
            current_qty_id = readState['qtyID']
            current_state_id = automaton_df.tail(1).index.item()
            print("State " + str(current_state_id) + " added to the automaton at " + current_time  + " (current state: " + str(current_state_id) + ")") 
            time_state = 0
            revisited_state = False
            return current_state_id
    

def addSlapsedTimeToState(configID, qtyID, timeToAdd):
    global time_state
    global automaton_df
    for i in range(len(automaton_df)):
        if (automaton_df.iloc[i]['configID'] == configID and automaton_df.iloc[i]['qtyID'] == qtyID):
            automaton_df.iloc[i]['slapsed_time'] = timeToAdd
            return True
    return False

def getAutomatonState(stateID):
    global automaton_df
    for i in range(len(automaton_df)):
        if (automaton_df.iloc[i]['stateID'] == stateID):
            return automaton_df.iloc[i]
    return -1

def getAutomatonStateByConfigID(configID):
    global automaton_df
    for i in range(len(automaton_df)):
        if (automaton_df.iloc[i]['configID'] == configID):
            return automaton_df.iloc[i]
    return -1

def getAutomatonStateByQtyID(qtyID):
    global automaton_df
    for i in range(len(automaton_df)):
        if (automaton_df.iloc[i]['qtyID'] == qtyID):
            return automaton_df.iloc[i]
    return -1

def getAutomatonStateByTransition(transition):
    global automaton_df
    for i in range(len(automaton_df)):
        if (automaton_df.iloc[i]['transition'] == transition):
            return automaton_df.iloc[i]
    return -1

def getAutomatonStateByConfigIDAndQtyID(configID, qtyID):
    global automaton_df
    for i in range(len(automaton_df)):
        if (automaton_df.iloc[i]['configID'] == configID and automaton_df.iloc[i]['qtyID'] == qtyID):
            return automaton_df.iloc[i]
    return pd.DataFrame()

def getIndexAutomatonStateByConfigIDAndQtyID(configID, qtyID):
    global automaton_df
    for i in range(len(automaton_df)):
        if (automaton_df.iloc[i]['configID'] == configID and automaton_df.iloc[i]['qtyID'] == qtyID):
            return i
    return -1

def checkAutomatonStateByConfigIDAndQtyID(configID, qtyID):
    global automaton_df
    for i in range(len(automaton_df)):
        if (automaton_df.iloc[i]['configID'] == configID and automaton_df.iloc[i]['qtyID'] == qtyID):
            return True
    return False

def getAutomatonStateByConfigIDAndTransition(configID, transition):
    global automaton_df
    for i in range(len(automaton_df)):
        if (automaton_df.iloc[i]['configID'] == configID and automaton_df.iloc[i]['transition'] == transition):
            return automaton_df.iloc[i]
    return -1

def getAutomatonStateByQtyIDAndTransition(qtyID, transition):
    global automaton_df
    for i in range(len(automaton_df)):
        if (automaton_df.iloc[i]['qtyID'] == qtyID and automaton_df.iloc[i]['transition'] == transition):
            return automaton_df.iloc[i]
    return -1

def addSecondQuery(query, secondsToAdd):
    start_index = query.index('start:') + 7
    stop_index = query.index('stop:') + 6

    start_time_str = query[start_index:query.index('Z', start_index)]
    stop_time_str = query[stop_index:query.index('Z', stop_index)]

    start_time = datetime.fromisoformat(start_time_str)
    stop_time = datetime.fromisoformat(stop_time_str)

    modified_start_time = (start_time + timedelta(seconds=secondsToAdd)).isoformat()
    modified_stop_time = (stop_time + timedelta(seconds=secondsToAdd)).isoformat()

    modified_query = query[:start_index] + modified_start_time + query[query.index('Z', start_index):stop_index] + modified_stop_time + query[query.index('Z', stop_index):]

    return modified_query

def expandQuery(query):
    start_index = query.index('start:') + 7
    stop_index = query.index('stop:') + 6

    start_time_str = query[start_index:query.index('Z', start_index)]
    stop_time_str = query[stop_index:query.index('Z', stop_index)]

    start_time = datetime.fromisoformat(start_time_str)
    stop_time = datetime.fromisoformat(stop_time_str)

    modified_start_time = (start_time - timedelta(seconds=1)).isoformat()
    modified_stop_time = (stop_time + timedelta(seconds=1)).isoformat()

    modified_query = query[:start_index] + modified_start_time + query[query.index('Z', start_index):stop_index] + modified_stop_time + query[query.index('Z', stop_index):]

    return modified_query

def getStateSlapsedTime(state_index):
    global automaton_df
    times = []
    if (automaton_df.empty == False):
        for i in range(len(automaton_df.iloc[state_index]['transition'])):
            if (automaton_df.iloc[state_index]['transition'][i].startswith("qty")):
                times.append(automaton_df.iloc[state_index]['slapsed_time'][i])
    return times

def predictionDetected(qty_objective, read_qty_id, goal_time):
    global config_time
    global time_state
    global current_state_id
    global current_qty_id
    global revisited_state
    slapsed_times_list = getStateSlapsedTime(current_state_id)

    if (revisited_state == True and len(slapsed_times_list) > 0):
        min_slapsed_time = min([x for x in getStateSlapsedTime(current_state_id)])
        if (min_slapsed_time > goal_time):
            revisited_state = False
            return True
    else:   
        return False
    
def getSuitableConfigurations(qty_objective, goal_time):
    global automaton_df
    global total_configs_df
    suitable_configs = []

    #Filter automaton_df by qtyID
    automaton_df_qty_obj = automaton_df[automaton_df['qtyID'] == qty_objective]
    #Sum slapsed_time values
    automaton_df_qty_obj['sum_slapsed_time'] = automaton_df_qty_obj['slapsed_time'].apply(lambda x: sum(x))
    #Sort by sum_slapsed_time
    automaton_df_qty_obj.sort_values(by=['sum_slapsed_time'], inplace=True, ascending=False)
    
    for i in range(len(automaton_df_qty_obj)):
        if ((automaton_df_qty_obj.iloc[i]['sum_slapsed_time'].min()) >= goal_time):
            suitable_configs.append(getConfigParams(automaton_df_qty_obj.iloc[i]['configID']))
    return suitable_configs

def networkReconfiguration(selectedConfig):
    global old_config_id
    print("Network reconfiguration started (ETA: 40 seconds)")
    subprocess.call(["/home/schempp/Documents/NokiaAdminCLI/setConfigMicroArguments.sh", selectedConfig[0], selectedConfig[1], selectedConfig[2], selectedConfig[3], selectedConfig[4], selectedConfig[5], selectedConfig[6], selectedConfig[7], selectedConfig[8], selectedConfig[9], selectedConfig[10], selectedConfig[11]])
    subprocess.call(["/home/schempp/Documents/NokiaAdminCLI/telit_reconnection.sh"])
    if (len(selectedConfig) == 13):
        addReconfiguration(old_config_id, selectedConfig[12], datetime.utcnow().isoformat())
        old_config_id = selectedConfig[12]
    else:
        old_config_id = getConfigIDhash(selectedConfig)
    print("Network reconfiguration completed")
    return True

def getConfigIDhash(config):
    return hashlib.sha256(''.join(config).encode('utf-8')).hexdigest()

def reconfigAlreadyTested(configID):
    global reconfigurations_df
    if (reconfigurations_df.empty == False):
        if (configID in reconfigurations_df['original_configID'].values):
            print("Selected config (" + configID + ") already tested")
            return True
    return False

def insertIntervalConfigQuery(interval_time_to_add):
    first_part = 'from(bucket: "schempp")|> range(start: -'
    second_part = 's)|> filter(fn: (r) => r["parameter"] == "dlMaxRetxThreshold" or r["parameter"] == "dlPollByte" or r["parameter"] == "dlPollPDU" or r["parameter"] == "dlTPollRetr" or r["parameter"] == "dlTProhib" or r["parameter"] == "dlTReassembly" or r["parameter"] == "ulMaxRetxThreshold" or r["parameter"] == "ulPollByte" or r["parameter"] == "ulPollPDU" or r["parameter"] == "ulTPollRetr" or r["parameter"] == "ulTProhib" or r["parameter"] == "ulTReassembly")|> sort()|> yield(name: "sort")'
    return first_part + str(interval_time_to_add) + second_part

def insertStartStopConfigQuery(start_time, stop_time):
    first_part = 'from(bucket: "schempp")|> range(start: '
    second_part = ', stop: '
    third_part = ')|> filter(fn: (r) => r["parameter"] == "dlMaxRetxThreshold" or r["parameter"] == "dlPollByte" or r["parameter"] == "dlPollPDU" or r["parameter"] == "dlTPollRetr" or r["parameter"] == "dlTProhib" or r["parameter"] == "dlTReassembly" or r["parameter"] == "ulMaxRetxThreshold" or r["parameter"] == "ulPollByte" or r["parameter"] == "ulPollPDU" or r["parameter"] == "ulTPollRetr" or r["parameter"] == "ulTProhib" or r["parameter"] == "ulTReassembly")|> sort()|> yield(name: "sort")'
    return first_part + start_time + second_part + stop_time + third_part

def insertIntervalKPIQuery(interval_time_to_add):
    first_part = 'from(bucket: "schempp")|> range(start: -'
    second_part = 's)|> filter(fn: (r) => r["_measurement"] == "kpis")|> filter(fn: (r) => r["_field"] == "delay" or r["_field"] == "jitter")|> aggregateWindow(every: '
    thrird_part = 's, fn: mean, createEmpty: false)|> yield(name: "mean")'
    return first_part + str(interval_time_to_add) + second_part + str(interval_time_to_add) + thrird_part

def insertStartStopKPIQuery(start_time, stop_time):
    first_part = 'from(bucket: "schempp")|> range(start: '
    second_part = ', stop: '
    third_part = ')|> filter(fn: (r) => r["_measurement"] == "kpis")|> filter(fn: (r) => r["_field"] == "delay" or r["_field"] == "jitter")|> aggregateWindow(every: '
    fourth_part = 's, fn: mean, createEmpty: false)|> yield(name: "mean")'
    return first_part + start_time + second_part + stop_time + third_part + str(config_time) + fourth_part


Load an automata for the pre-learning phase

In [ ]:
import ast

# Define the conversion function using ast.literal_eval()
def convert_to_string_list(string_data):
    return ast.literal_eval(string_data)

def convert_to_int_list(string_data):
    # Handle list-like string case
    if string_data.startswith("[") and string_data.endswith("]"):
        string_data = string_data[1:-1]  # Remove brackets
        return [int(x) for x in string_data.split(",")]  # Convert to integer list
    else:
        return [int(string_data)]  # Convert as usual

def convert_to_int(string_data):
    # Handle list-like string case
    if string_data.startswith("'") and string_data.endswith("'"):
        string_data = string_data[1:-1]  # Remove brackets
        print("Covertido: " + string_data)
        return int(string_data)  # Convert to integer
    else:
        return int(string_data)  # Convert as usual
    
def convert_to_float(string_data):
    return float(string_data)


# Read the CSV file and apply the conversion function to the desired column(s)
total_configs_df = pd.read_csv('./final_automatons_v1.15/4d/total_configs_10s_2023-06-03T12:37:00Z_2023-06-07T12:37:00Z.csv', converters={'configID': str})
automaton_df = pd.read_csv('./final_automatons_v1.15/4d/automaton_10s_2023-06-03T12:37:00Z_2023-06-07T12:37:00Z.csv', converters={'qtyID': int, 'configID': str, 'transition': convert_to_string_list, 'slapsed_time': convert_to_int_list})


Apply the initial configuration to the 5G Network (Nokia RAN) with the parameters selected

In [ ]:
baseline_config_ID1_23jun_12_44h = ['t6', 'kB8', 'p24576', 'ms500', 'ms800', 'ms170', 't2', 'mB14', 'p256', 'ms120', 'ms250', 'ms55']
baseline_config_ID2_27jun_16_16h = ['t3', 'kB6500', 'p20480', 'ms140', 'ms90', 'ms45', 't2', 'mB40', 'p256', 'ms60', 'ms130', 'ms90']
baseline_config_ID3_19jun_11_36h = ['t32', 'kB8', 'p1024', 'ms70', 'ms100', 'ms10', 't32', 'kB25', 'p32768', 'ms30', 'ms300', 'ms120']
baseline_config_ID4_28jun_19_51h = ['t32', 'mB40', 'p40960', 'ms40', 'ms40', 'ms180', 't32', 'kB1000', 'p1024', 'ms170', 'ms160', 'ms170']

initial_config = baseline_config_ID1_23jun_12_44h
networkReconfiguration(initial_config)


Parameters to be defined by the mobile operator (or user)

In [ ]:
# qty ranges definition 
#data = {'qty': [-18, -15, -5, 1, 2, 3, 4], 'min_delay': [10000000, 0, 10000000, 0, 7000000, 0, 7000000], 'max_delay': [2000000000000, 10000000, 2000000000000, 7000000, 10000000, 7000000, 10000000], 'min_jitter': [0, 10000000, 10000000, 0, 0, 1000000, 1000000], 'max_jitter': [10000000, 2000000000000, 2000000000000, 1000000, 1000000, 10000000, 10000000]}
data = {'qty': [-30, -18, -8, -1, 1, 2, 12, 13], 'min_delay': [12000000, 10000000, 10000000, 0, 0, 0, 7000000, 7000000], 'max_delay': [2000000000000, 12000000, 12000000, 7000000, 7000000, 7000000, 10000000, 10000000], 'min_jitter': [10000000, 8000000, 1000000, 8000000, 0, 1000000, 0, 1000000], 'max_jitter': [2000000000000, 10000000, 5000000, 10000000, 1000000, 5000000, 1000000, 5000000]}
qty_df = pd.DataFrame(data)

# Loop interval time: time (in seconds) for the operation loop
# note that the results of KPIs will be the mean during this period; lower values result in a fine-grained automata and viceversa
interval_time = 10
# qty objective; it depends on the application requirements
qty_objective = np.int64(1)
# Time (in seconds) to consider a network configuration as suitable for reconfiguration;
# note that higher values result in likely better configurations but probably less posibilities 
goal_time = 600
# Number of operation cycles to be consider as stability period after a reconfiguration; it depends on the interval time
# stability time (s) = interval_time (s) * cycles_for_stability 
cycles_for_stability = 18

Initialization of variables and queries

In [ ]:
# Initialization of variables
old_config_id = getConfigIDhash(initial_config)
current_cycle = 0
objective_reached = True

#Downlink and Uplink query
dl_ul_query = insertIntervalConfigQuery(interval_time*3)

#Combined query (delay and jitter)
combined_query = insertIntervalKPIQuery(interval_time)

AutomAdapt loop execution

In [ ]:
while(True):

    #Get downlink and uplink configs
    df_dl_ul_result = query_api.query_data_frame(query=dl_ul_query)
    #Get delay and jitter
    df_combined_result = query_api.query_data_frame(query=combined_query)

    #Combine kpis (mean)
    df_delay = pd.DataFrame()
    df_jitter = pd.DataFrame()
    if (df_combined_result.empty == False):
        df_kpis_result = df_combined_result.groupby(['_field']).mean(numeric_only=True)
        df_delay = df_kpis_result[df_kpis_result.index == 'delay']
        df_jitter = df_kpis_result[df_kpis_result.index == 'jitter']

    #Get qty ID (just read)
    if (df_delay.empty == False and df_jitter.empty == False):
        read_qty_id = getqtyID(df_delay['_value'].values[0], df_jitter['_value'].values[0])
        
    #Get config ID (just read)
    if (df_dl_ul_result.empty == False and len(df_dl_ul_result.index) == 12):
        df_aux = df_dl_ul_result.drop(columns=['_start', '_stop', '_time', '_measurement', '_field', 'result', 'table', 'parameter'])
        row = df_aux['_value'].tolist()
        read_config_id = getConfigIDhash(row)
        addConfig(row, read_config_id)
    
    #Check current state and add it to the automaton if it is not already there
    read_state = {"configID": read_config_id, "qtyID": read_qty_id, "transition": [""], "slapsed_time": [0]}
    manageAutomatonState(read_state)

    #Get current state
    print("The automaton has " + str(len(automaton_df)) + " states")
    print("Current automaton state " + str(current_state_id) + " --> configID = " + str(read_config_id) + " / qtyID = " + str(read_qty_id) + "\n")
    print("Objective qtyID = " + str(qty_objective) + "\n")

    # Check the qty objective
    if (read_qty_id != qty_objective):
        current_cycle = current_cycle + 1
    else:
        current_cycle = 0
    # Check the stability period
    if (current_cycle >= cycles_for_stability):
        objective_reached = False

    # Check if a reconfiguration is needed due to a prediction or the qty objective has not been reached
    if (predictionDetected(qty_objective, read_qty_id, goal_time) == True or objective_reached == False):
        print("Reconfiguration needed")
        suitableConfigs = getSuitableConfigurations(qty_objective, goal_time)
        if (len(suitableConfigs) > 0):
            print("Suitable configurations found")
            for i in range(len(suitableConfigs)):
                selectedConfig = suitableConfigs[i]
                if ((selectedConfig[12] != current_config_id) and (reconfigAlreadyTested(selectedConfig[12]) == False)):
                    networkReconfiguration(selectedConfig)
                    current_cycle = 0
                    objective_reached = True
                    break
            print("[WARNING] Any configuration has been selected...decreasing goal time")
            if (goal_time > 0):
                goal_time = goal_time - 10
            else:
                print("[FATAL ERROR] No more goal time to decrease; the system is not able to reach the objective")
                break
        else:
            print("[ERROR] No suitable configuration found")

    # Update the time state
    time_state = time_state + interval_time
    # Wait for the next loop
    time.sleep(interval_time)
    # Clear the output
    clear_output(wait=True)


Store the results obtained

In [ ]:
total_configs_df.to_csv('./final_automatons_v1.15/Reconfigurations/total_configs_Reconfigurationsv16.csv', index=False)
automaton_df.to_csv('./final_automatons_v1.15/Reconfigurations/automaton_Reconfigurationsv16.csv', index=False)
reconfigurations_df.to_csv('./final_automatons_v1.15/Reconfigurations/reconfigurations_Reconfigurationsv16.csv', index=False)

Create network graph to be plotted using Flourish tool

In [ ]:
network_graph_df = pd.DataFrame(columns=['source', 'destination'])

for i in range(len(automaton_df)-1):
    for j in range(len(automaton_df.at[i, 'transition'])):
        network_graph_df = network_graph_df.append({'source': i, 'destination': automaton_df.at[i, 'transition'][j].split('-')[1]}, ignore_index=True)

network_graph_df['weight'] = automaton_df['slapsed_time'].apply(lambda x: sum(x))

network_graph_df.to_csv('./final_automatons_v1.15/network_graph_10s_2023-06-03T12:37:00Z_2023-06-07T14:51:00Z.csv', index=False)